In [1]:
import os
import torch
import yaml
from addict import Dict
from model_lightning_seg import MyModel

# === Configura tus rutas ===
ckpt_dir = "/data/GitHub/Breast-AI-model/experiment_5_giancarlo"
config_path = "/data/GitHub/Breast-AI-model/src/default_config_train_seg.yaml"
onnx_out_dir = "./onnx_models"
os.makedirs(onnx_out_dir, exist_ok=True)

# === Carga el config YAML ===
with open(config_path, "r") as f:
    conf = Dict(yaml.safe_load(f))

# === Encuentra los .ckpt ===
ckpts = sorted([
    os.path.join(ckpt_dir, f)
    for f in os.listdir(ckpt_dir)
    if f.endswith(".ckpt")
])

if not ckpts:
    raise FileNotFoundError(f"No .ckpt found in {ckpt_dir}")

# === Exporta a ONNX con Lightning ===
for i, ckpt_path in enumerate(ckpts):
    print(f"📦 Exportando modelo {i+1}/{len(ckpts)}: {ckpt_path}")
    
    model = MyModel.load_from_checkpoint(
        checkpoint_path=ckpt_path,
        model_opts=conf.model_opts,
        train_par=conf.train_par,
        strict=False,
    )
    model.eval()

    # Dummy input: (1, 1, 128, 128, 128)
    dummy_input = torch.randn(1, 1, 128, 128, 128, dtype=torch.float32)

    # Ruta de salida ONNX
    onnx_path = os.path.join(onnx_out_dir, f"model_fold{i}.onnx")

    # Exportar usando Lightning
    model.to_onnx(
        file_path=onnx_path,
        input_sample=dummy_input,
        export_params=True,
        opset_version=17,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    )

    print(f"✅ Guardado: {onnx_path}\n")

print("🎉 Conversión a ONNX completada.")


📦 Exportando modelo 1/10: /data/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_1.ckpt


/data/GitHub/Breast-AI-model/src/models_arq/att_unet_seg.py:45: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x_up.shape[2:] != y.shape[2:]:
/home/exx/anaconda3/envs/ligthning/lib/python3.12/site-packages/torch/onnx/symbolic_helper.py:1460: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'instance_norm' is set to train=True. Exporting with train=True.
  warnings.warn(


✅ Guardado: ./onnx_models/model_fold0.onnx

📦 Exportando modelo 2/10: /data/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_10.ckpt
✅ Guardado: ./onnx_models/model_fold1.onnx

📦 Exportando modelo 3/10: /data/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_2.ckpt
✅ Guardado: ./onnx_models/model_fold2.onnx

📦 Exportando modelo 4/10: /data/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_3.ckpt
✅ Guardado: ./onnx_models/model_fold3.onnx

📦 Exportando modelo 5/10: /data/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_4.ckpt
✅ Guardado: ./onnx_models/model_fold4.onnx

📦 Exportando modelo 6/10: /data/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_5.ckpt
✅ Guardado: ./onnx_models/model_fold5.onnx

📦 Exportando modelo 7/10: /data/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_6.ckpt
✅ Guardado: ./onnx_models/model_fold6.onnx

📦 Exportando modelo 8/10: /data/GitHub/Breast-AI-model/experi